In [38]:
from pynq import Overlay
from pynq.lib import AxiGPIO
import time


overlay = Overlay("puf.bit")
overlay.download()

print (overlay.ip_dict.keys())

print("done")

dict_keys(['axi_gpio_0', 'axi_gpio_1', 'axi_gpio_2', 'axi_gpio_3', 'axi_gpio_4', 'axi_gpio_5', 'processing_system7_0'])
done


In [58]:
from pynq import Overlay
from pynq.lib import AxiGPIO
import time


overlay = Overlay("puf.bit")

sel0_gpio = overlay.axi_gpio_0.channel1
sel1_gpio = overlay.axi_gpio_1.channel1
sel2_gpio = overlay.axi_gpio_2.channel1
sel3_gpio = overlay.axi_gpio_3.channel1
out = overlay.axi_gpio_4.channel1
pulse = overlay.axi_gpio_5.channel1


CRP_TABLE = [
    [0xDEADBEEF, 0xCAFEBABE, 0x12345678, 0x0BADF00D],
    [0x0F0F0F0F, 0xF0F0F0F0, 0xAAAAAAAA, 0x55555555],
    [0x11111111, 0x2A22B434, 0x33B1C5FA, 0x4d44F4A4],
    [0x89ABCDEF, 0x76543210, 0xFEDCBA98, 0x01234567],]

while len(CRP_TABLE) < 128:
    base = len(CRP_TABLE)
    CRP_TABLE.append([
        base | 0x01010101,
        base | 0x02020202,
        base | 0x03030303,
        base | 0x04040404
    ])


key_bits = []

print("Generating stable 128-bit PUF key...\n")

for idx, challenge in enumerate(CRP_TABLE):
    sel0_gpio.write(challenge[0], 0xFFFFFFFF)
    sel1_gpio.write(challenge[1], 0xFFFFFFFF)
    sel2_gpio.write(challenge[2], 0xFFFFFFFF)
    sel3_gpio.write(challenge[3], 0xFFFFFFFF)
    
    time.sleep(0.0001) 

    pulse.write(1,0x1)
    time.sleep(0.001) 
    pulse.write(0,0x1)
    time.sleep(0.0001)

    bit = out.read()
    key_bits.append(str(bit))

key_binary = ''.join(key_bits)
key_hex = hex(int(key_binary, 2))[2:].zfill(32)  

print(f"PUF Raw Key Bits (128):\n{key_binary}")
print(f"\n128-bit PUF Key (HEX): {key_hex}")

Generating stable 128-bit PUF key...

PUF Raw Key Bits (128):
01111111111111101111111010011111111101101001111110010111111101101111111010011011100110111111011010011011111011101110111010011011

128-bit PUF Key (HEX): 7ffefe9ff69f97f6fe9b9bf69beeee9b
